In [20]:
#adapted after https://www.kaggle.com/sentdex/data-science-bowl-2017/first-pass-through-data-w-3d-convnet

#NOT FINISHED
#ZADEVA JE V DELU MA NACELOMA ZDEJ JE SKORI TU TU

import dicom
import os
import matplotlib.pyplot as plt
import cv2
import numpy as np
import csv
import math

In [21]:
import math

def chunks(l, n):
    for i in range(0, len(l), n):
        yield l[i:i + n]

def mean(l):
    return sum(l) / len(l)

IMG_PX_SIZE = 160
HM_SLICES = 24

In [10]:
def process_data(patient,img_px_size=160, hm_slices=24):
    
    slices = [dicom.read_file(folder + '/' + s) for s in os.listdir(folder)]
    slices.sort(key = lambda x: int(x.ImagePositionPatient[2]))
    
    #info about picture
    #print(slices[0])
    
    #print array shape
    #print(slices[0].pixel_array.shape, len(slices)) 
    
    #show a slice of choice
    #plt.imshow(slices[125].pixel_array)
    #plt.show()
    
    #resizing to 150*150*20
    new_slices = []

    slices = [cv2.resize(np.array(each_slice.pixel_array),(img_px_size,img_px_size)) for each_slice in slices]

    chunk_sizes = math.ceil(len(slices) / hm_slices)

    for slice_chunk in chunks(slices, chunk_sizes):
        slice_chunk = list(map(mean, zip(*slice_chunk)))
        new_slices.append(slice_chunk)
        
    if len(new_slices) == hm_slices-1:
        new_slices.append(new_slices[-1])

    if len(new_slices) == hm_slices-2:
        new_slices.append(new_slices[-1])
        new_slices.append(new_slices[-1])

    if len(new_slices) == hm_slices+2:
        new_val = list(map(mean, zip(*[new_slices[hm_slices-1],new_slices[hm_slices],])))
        del new_slices[hm_slices]
        new_slices[hm_slices-1] = new_val

    if len(new_slices) == hm_slices+1:
        new_val = list(map(mean, zip(*[new_slices[hm_slices-1],new_slices[hm_slices],])))
        del new_slices[hm_slices]
        new_slices[hm_slices-1] = new_val
    
    #showing resized pictures
    #fig = plt.figure()
    #for num,each_slice in enumerate(new_slices):
    #    y = fig.add_subplot(4,5,num+1)
    #    y.imshow(each_slice, cmap='gray')
    #plt.show()
    
    return np.array(new_slices)

In [11]:
data_dir = '../4D-Lung/'
#patient 0
path = data_dir + '100_HM10395/1.3.6.1.4.1.14519.5.2.1.6834.5010.465205689126985052184293614571/'

folders = [x[0] for x in os.walk(path)]
del folders[0]
#folders[:10]

data = []
data_index = [8,3,1,6,5,2,7,9,4,0]

for num,folder in enumerate(folders[:10]):
    img_data=process_data(folder,img_px_size=IMG_PX_SIZE, hm_slices=HM_SLICES)
    #print(img_data.shape)
    data.append(img_data)

data_sorted = [x for y, x in sorted(zip(data_index,data))]
np.save('data-{}-{}-{}.npy'.format(IMG_PX_SIZE,IMG_PX_SIZE,HM_SLICES), data_sorted)

In [13]:
def resize_seg(data, img_px_size=160, hm_slices=24):
    new_slices = []
    
    data_seg = []
    for num in range(141):
        #print(data.shape)
        data1 = np.array(data)[:,:,num]
        data_seg.append(cv2.resize(data1,(img_px_size,img_px_size)))

    chunk_sizes = math.ceil(142 / hm_slices)

    for slice_chunk in chunks(data_seg, chunk_sizes):
        slice_chunk = list(map(mean, zip(*slice_chunk)))
        new_slices.append(slice_chunk)
        
    if len(new_slices) == hm_slices-1:
        new_slices.append(new_slices[-1])

    if len(new_slices) == hm_slices-2:
        new_slices.append(new_slices[-1])
        new_slices.append(new_slices[-1])

    if len(new_slices) == hm_slices+2:
        new_val = list(map(mean, zip(*[new_slices[hm_slices-1],new_slices[hm_slices],])))
        del new_slices[hm_slices]
        new_slices[hm_slices-1] = new_val

    if len(new_slices) == hm_slices+1:
        new_val = list(map(mean, zip(*[new_slices[hm_slices-1],new_slices[hm_slices],])))
        del new_slices[hm_slices]
        new_slices[hm_slices-1] = new_val
    
    #showing resized pictures
    #fig = plt.figure()
    #for num,each_slice in enumerate(new_slices):
    #    y = fig.add_subplot(4,5,num+1)
    #    y.imshow(each_slice, cmap='gray')
    #plt.show()
    
    return np.array(new_slices)

In [ ]:
data_seg_dir = '../patient0-segmented/'

files = [x for x in os.listdir(data_seg_dir)]

print(files)

data_seg = []
for num,file in enumerate(data_seg_dir):
    if num>=10:
        break
    print(data_seg_dir + str(num*10) + '-cancer.csv')
    seg_data = np.genfromtxt(data_seg_dir + str(num*10) + '-cancer.csv',delimiter=",")
    seg_data = seg_data.reshape((512,512,142))
    #print(seg_data.shape)
    seg_data = resize_seg(seg_data)
    data_seg.append(seg_data)

np.save('segdata-{}-{}-{}.npy'.format(IMG_PX_SIZE,IMG_PX_SIZE,HM_SLICES), data_seg)

['0-cancer.csv', '10-cancer.csv', '20-cancer.csv', '30-cancer.csv', '40-cancer.csv', '50-cancer.csv', '60-cancer.csv', '70-cancer.csv', '80-cancer.csv', '90-cancer.csv']
../patient0-segmented/0-cancer.csv
../patient0-segmented/10-cancer.csv
../patient0-segmented/20-cancer.csv
../patient0-segmented/30-cancer.csv
../patient0-segmented/40-cancer.csv
../patient0-segmented/50-cancer.csv
../patient0-segmented/60-cancer.csv
../patient0-segmented/70-cancer.csv
../patient0-segmented/80-cancer.csv
../patient0-segmented/90-cancer.csv


In [26]:
print(len(data_seg))
print(len(data_seg[0]))
np.save('segdata-{}-{}-{}.npy'.format(IMG_PX_SIZE,IMG_PX_SIZE,HM_SLICES), data_seg)

2
24


In [30]:
data = np.load('data-160-160-24.npy')
seg_data = np.load('segdata-160-160-24.npy')

print(seg_data.shape)

data = np.expand_dims(data, axis=1)
seg_data = np.expand_dims(seg_data, axis=1)
print(seg_data.shape)


train_data = data[:-2]
validation_data = data[-2:]
train_seg = seg_data[:-2]
validation_seg = seg_data[-2:]

print(train_seg.shape)
for x in train_seg:
    print(x.shape)

(3, 24, 160, 160)
(3, 1, 24, 160, 160)
(1, 1, 24, 160, 160)
(1, 24, 160, 160)
